# Create Agents to Research, Write and Edit a Letter of Correspondence

## Setting a Goal

Write a letter of correspondence on an issue for a country leader.

In [1]:
# Install required libraries
!pip install -q 'crewai[tools]' langchain-groq duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.1/66.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 26.0 MB/s eta 0:00:00
  Preparing metadata (setup

In [2]:
# Import modules
from crewai import Agent, Task, Crew
from crewai_tools import tool
from langchain_groq import ChatGroq
from langchain_community.tools import DuckDuckGoSearchRun
from google.colab import userdata

In [3]:
# Access Colab secret keys
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

In [4]:
# Initialize ChatGroq class with a model
llm = ChatGroq(
    temperature=0,
    groq_api_key=GROQ_API_KEY,
    model_name="mixtral-8x7b-32768"
)

In [5]:
# Use DuckDuckGo Search component
@tool('DuckDuckGoSearch')
def search(query: str):
    """
    Search the web using DuckDuckGo with an input query.
    """
    return DuckDuckGoSearchRun().run(query)

## Creating Agents

### Agent: Researcher

In [6]:
researcher = Agent(
    role="Senior Researcher",
    goal="Collect and analyze relevant data, literature, and information on {issue}, {sender}, and {recipient}.",
    backstory="You work at the office of a country leader: {sender}. "
              "You are writing a letter of correspondence on {issue} from {sender} to {recipient}. "
              "You are an expert at reviewing existing research and analyzing data. "
              "You collect information that provides valuable insights and recommendations. "
              "Your work is the basis for the Senior Writer to write a letter of correspondence on {issue} from {sender} to {recipient}.",
    allow_delegation=False,
    verbose=True,
    llm=llm,
    tools=[search]
)

### Agent: Writer

In [7]:
writer = Agent(
    role="Senior Writer",
    goal="Write a professional, clear, and effective letter of correspondence on {issue} from {sender} to {recipient}.",
    backstory="You work at the office of a country leader: {sender}. "
              "You are writing a letter of correspondence on {issue} from {sender} to {recipient}. "
              "You are an expert at writing letters that require careful consideration and attention to detail. "
              "You base your writing on the work of the Senior Researcher, who provides an outline, relevant context and background information about {issue}, {sender}, and {recipient}. "
              "You follow the main objectives and direction of the outline, as provided by the Senior Researcher.",
    allow_delegation=True,
    verbose=True,
    llm=llm
)

### Agent: Editor

In [8]:
editor = Agent(
    role="Senior Editor",
    goal="Edit the given letter of correspondence on {issue} to align with the writing style of the office of a country leader: {sender}.",
    backstory="You are a Senior Editor who receives a letter of correspondence from the Senior Writer. "
              "You are an expert at proofreading, editing and revising letters of correspondence. "
              "Your goal is to review the letter of correspondence to ensure that it follows research and writing best practices, "
              "and provides balanced viewpoints when providing opinions or assertions.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

## Creating Tasks

### Task: Research

In [9]:
research = Task(
    description=(
        "1. Prioritize the {sender}'s opinions and statements on {issue} that are relevant to {recipient}.\n"
        "2. Define the purpose of the letter, and determine the main message to be conveyed to {recipient}.\n"
        "3. Identify the specific issue, concern, or proposal to be addressed.\n"
        "4. Provide relevant context and background information to help the {recipient} understand the {issue}.\n"
        "6. Develop a detailed letter outline including a formal greeting, purpose and main message, context and background information, specific proposal or request, solutions or alternatives, and a formal closing."
    ),
    expected_output="A comprehensive outline for a letter of correspondence.",
    agent=researcher
)

### Task: Write

In [10]:
write = Task(
    description=(
        "1. Use the outline received from the Senior Researcher to write a letter of correspondence on {issue} from {sender} to {recipient}.\n"
        "2. Structure the letter by organizing the information from the Senior Researcher into clear and concise sections.\n"
        "3. Ensure the sections include a formal greeting, purpose and main message, context and background information, specific proposal or request, solutions or alternatives, and a formal closing.\n"
        "4. Use a clear and concise writing style, avoiding jargon and technical terms that may be unfamiliar to {recipient}.\n"
        "5. Choose the right tone to convey: formal, informal, diplomatic, or persuasive.\n"
        "6. Be mindful of the {recipient}'s cultural and linguistic background.\n"
        "7. Use a formal salutation to address the {recipient} with the proper title and honorifics.\n"
        "8. Ensure the section on context and background information is concise and focused on the key points.\n"
        "9. Be specific about what the {sender} is asking for, and why it is important.\n"
        "10. If necessary, offer potential solutions or alternatives to the {issue}, and provide additional information or clarification.\n"
        "11. Use a formal closing, such as 'Sincerely' or 'Yours faithfully' to close the letter.\n"
        "12. Include the {sender}'s full name, spaced 3 lines after the closing."
    ),
    expected_output="A professional, clear, and effective letter of correspondence in markdown format, ready for review by the Senior Editor.",
    agent=writer
)

### Task: Edit

In [11]:
edit = Task(
    description=(
        "1. Review the letter of correspondence received from the Senior Writer, for clarity, accuracy, and relevance.\n"
        "2. Ensure the tone and language are consistent throughout the letter.\n"
        "3. Proofread the letter for errors in grammar, spelling, and punctuation.\n"
        "4. Ensure that the formatting is consistent and easy to read.\n"
        "5. Finalize the letter and prepare it for transmission to the appropriate diplomatic channels."
    ),
    expected_output="A professional, clear, and effective letter of correspondence in markdown format, ready for transmission to diplomatic channels.",
    agent=editor
)

## Creating the Crew

- **Note**: *In this crew*, the Writer agent can delegate tasks to other agents.

In [12]:
crew = Crew(
    agents=[researcher, writer, editor],
    tasks=[research, write, edit],
    verbose=2,
    # memory=True
)

## Running the Crew

In [13]:
issue = "peace"
sender = "Joseph Robinette Biden, Jr."
recipient = "Vladimir Vladimirovich Putin"

In [14]:
inputs = {
    "issue": issue,
    "sender": sender,
    "recipient": recipient
}

In [15]:
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Senior Researcher
 [INFO]: == Starting Task: 1. Prioritize the Joseph Robinette Biden, Jr.'s opinions and statements on peace that are relevant to Vladimir Vladimirovich Putin.
2. Define the purpose of the letter, and determine the main message to be conveyed to Vladimir Vladimirovich Putin.
3. Identify the specific issue, concern, or proposal to be addressed.
4. Provide relevant context and background information to help the Vladimir Vladimirovich Putin understand the peace.
6. Develop a detailed letter outline including a formal greeting, purpose and main message, context and background information, specific proposal or request, solutions or alternatives, and a formal closing.


> Entering new CrewAgentExecutor chain...
To complete this task, I need to gather information on Joseph Robinette Biden, Jr.'s opinions and statements on peace, as well as relevant context and background information. I will use DuckDuckGoSearch to find this information.

Action: Du

- Display the results of the execution as markdown in the notebook.

In [16]:
from IPython.display import Markdown
Markdown(result)

---

[Office of Joseph Robinette Biden, Jr.]

[Date]

[Recipient's Name]
[Recipient's Position]
[Recipient's Organization]
[Recipient's Address]

Dear [Recipient's Name],

I am writing to express our administration's unwavering commitment to fostering peace, security, and stability across the globe. We believe that a world free from conflict and strife is essential for the prosperity and well-being of all nations and their people.

In our pursuit of peace, we recognize the importance of multilateralism, diplomacy, and dialogue. We are committed to working closely with our international partners, including your esteemed organization, to address the root causes of conflict, promote social cohesion, and strengthen institutions that uphold the rule of law.

We are particularly concerned about the ongoing conflicts in [specific regions or countries]. The suffering and displacement of innocent civilians, the destruction of infrastructure, and the disruption of social and economic development are unacceptable. We urge all parties to cease hostilities, engage in good-faith negotiations, and commit to a peaceful resolution of their differences.

Furthermore, we are deeply alarmed by the proliferation of weapons of mass destruction and the escalating tension in various regions. We reaffirm our commitment to nuclear non-proliferation and disarmament and call on all nations to adhere to their treaty obligations and international law. We also urge all parties to exercise restraint, refrain from provocative actions, and engage in confidence-building measures to reduce the risk of accidental escalation.

To promote peace and stability, we must also address the underlying drivers of conflict, such as poverty, inequality, and marginalization. We are committed to working with our international partners to promote inclusive and sustainable development, expand access to education and economic opportunities, and protect human rights and fundamental freedoms. We believe that a world where every individual has the opportunity to thrive is a world that is more peaceful, secure, and prosperous.

In conclusion, we reaffirm our commitment to working with our international partners, including your esteemed organization, to promote peace, security, and stability. We believe that through multilateralism, diplomacy, and dialogue, we can overcome the challenges that threaten our collective peace and security and build a better future for all.

Thank you for your attention to this matter. We look forward to continuing our collaboration and partnership in the pursuit of a more peaceful and prosperous world.

Sincerely,

[Your Name]
[Your Position]
[Your Office]